In [1]:
import random

class Player:
    def __init__(self, name, balance=1500):
        """Initialize a player with a name and a starting balance of $1500."""
        self.name = name
        self.balance = balance
        self.position = 0
        self.properties = []

class Monopoly:
    def __init__(self):
        """Initialize the Monopoly game with predefined properties and spaces on an extended board."""
        self.players = []
        self.owned_properties = [None] * 36  # Board with 36 spaces
        self.board = [
            ("GO", "No", None),
            ("Mediterranean Avenue", 60, 100),
            ("Community Chest", "No", None),
            ("Baltic Avenue", 60, 100),
            ("Income Tax", "No", None),
            ("Reading Railroad", 200, 200),
            ("Oriental Avenue", 100, 150),
            ("Chance", "No", None), 
            ("Vermont Avenue", 100, 150),
            ("Connecticut Avenue", 120, 150),
            ("Boardwalk", 400, 400)  # More properties can be added as needed
        ]
        self.max_rounds = 100  # Maximum rounds to prevent infinite game

    def setup_players(self):
        """Setup players for the game, ensuring 2-8 players participate."""
        try:
            num_players = int(input("How many players will be playing Monopoly (2-8)?: "))
            if num_players < 2 or num_players > 8:
                print("ERROR! Must have 2 to 8 players.")
                return False

            for i in range(num_players):
                name = input(f"Player {i+1}, please enter your name: ")
                self.players.append(Player(name))
            return True
        except ValueError:
            print("ERROR! Please enter a valid number.")
            return False

    def print_starting_balances(self):
        """Print starting balances for all players."""
        for player in self.players:
            print(f"{player.name}: ${player.balance}")

    def roll_dice(self):
        """Roll two dice to determine the number of steps a player moves."""
        return random.randint(1, 6) + random.randint(1, 6)

    def move_player(self, player, steps):
        """Move the player around the board based on dice roll."""
        player.position = (player.position + steps) % len(self.board)

    def setup_community_chest(self):
        """Prepare and shuffle Community Chest cards."""
        self.community_chest_cards = [
            ("Advance to Go (Collect $200)", 200),
            ("Bank error in your favor. Collect $200", 200),
            ("Doctor's fees. Pay $50", -50),
            # Additional cards can be included here
        ]
        random.shuffle(self.community_chest_cards)

    def draw_community_chest_card(self, player):
        """Draw a Community Chest card and apply its effect to the player."""
        card, effect = self.community_chest_cards.pop(0)
        print(f"{player.name} drew a Community Chest card: {card}")
        player.balance += effect
        self.community_chest_cards.append((card, effect))

    def buy_property(self, player, property_index):
        """Allow players to purchase unowned properties they land on."""
        property_name, property_price, _ = self.board[property_index]
        if player.balance >= property_price:
            player.balance -= property_price
            player.properties.append(property_name)
            self.owned_properties[property_index] = player
            print(f"{player.name} bought {property_name} for ${property_price}.")
        else:
            print(f"{player.name} cannot afford {property_name}.")

    def play_game(self):
        """Play the game until only one player is not bankrupt or a maximum number of rounds are reached."""
        if not self.setup_players():
            return
        self.print_starting_balances()
        self.setup_community_chest()

        current_round = 0
        while len([p for p in self.players if p.balance > 0]) > 1 and current_round < self.max_rounds:
            for player in self.players:
                if player.balance <= 0:
                    continue

                print(f"\n{player.name}'s turn:")
                input("Press Enter to roll the dice...")
                dice_roll = self.roll_dice()
                print(f"{player.name} rolled {dice_roll}.")
                self.move_player(player, dice_roll)
                current_property = self.board[player.position]

                print(f"{player.name} landed on {current_property[0]}.")

                if current_property[1] == "No":  # Non-purchasable spaces
                    if current_property[0] == "Community Chest":
                        self.draw_community_chest_card(player)
                    elif current_property[0] == "Chance":
                        # Placeholder for chance card logic
                        self.draw_community_chest_card(player)  # Temporarily using Community Chest logic
                    elif current_property[0] == "Income Tax":
                        tax_amount = min(200, int(player.balance * 0.1))  # 10% income tax or $200, whichever is less
                        print(f"{player.name} pays ${tax_amount} in income tax.")
                        player.balance -= tax_amount
                    # Add more special cases for other spaces like "Go to Jail", etc.
                else:  # Purchasable properties
                    property_index = player.position
                    if self.owned_properties[property_index] is None:
                        print(f"{current_property[0]} is available for purchase.")
                        response = input("Do you want to buy it? (yes/no): ").lower()
                        if response == "yes":
                            self.buy_property(player, property_index)
                        else:
                            print(f"{player.name} decided not to buy {current_property[0]}.")

                    elif self.owned_properties[property_index] != player:
                        owner = self.owned_properties[property_index]
                        rent = current_property[2] * 0.1  # Simplified rent: 10% of property price
                        print(f"{current_property[0]} is owned by {owner.name}. {player.name} pays ${rent} in rent.")
                        player.balance -= rent
                        owner.balance += rent
                        if player.balance <= 0:
                            print(f"{player.name} has gone bankrupt!")

            current_round += 1

        # End of game summary
        remaining_players = [p for p in self.players if p.balance > 0]
        if len(remaining_players) == 1:
            print(f"\nGame over! {remaining_players[0].name} wins the game!")
        else:
            print(f"\nGame over after {self.max_rounds} rounds. It's a draw among the remaining players.")

if __name__ == "__main__":
    print("Welcome to Monopoly! Win by bankrupting the other players!\n")
    game = Monopoly()
    game.play_game()


Welcome to Monopoly! Win by bankrupting the other players!

